# QSVT on Hamiltonian matrices

*   The `qm9_huckel` DataFrame is now prepared for subsequent quantum chemistry analyses, such as calculating eigenvalues, or for use in machine learning models that require molecular electronic structure information.
*   Further investigation could involve analyzing the distribution of matrix sizes in the 'H_pi' column to understand the complexity of π-systems across the dataset, and explicitly documenting the convention of empty lists for non-pi systems.

In [5]:
# install prereqs
!pip install qiskit
!pip install rdkit
!pip install pennylane pyqsp
!pip install pyqsp

In [10]:
# import tools
import ast
import pandas as pd
import numpy as np
import pennylane as qml
from rdkit import Chem
from pyqsp.angle_sequence import QuantumSignalProcessingPhases

In [15]:
# Load the QM9 Hückel Hamiltonian data
qm9_huckel = pd.read_csv('../qm9_hamiltonians.csv')

qm9_huckel.head()

,smiles,H_pi,pi_atoms
0,C,[],[]
1,N,[],[]
2,O,[],[]
3,C#C,"[[-0.0, -1.0], [-1.0, -0.0]]","[0, 1]"
4,C#N,"[[-0.0, -1.0], [-1.0, -0.5]]","[0, 1]"


#### Normalize

In [16]:
# Normalize Hamiltonian to ||H||_2 = 1 for QSVT block-encoding
def normalize_hamiltonian(H):
    if H is None or H.size == 0:
        return None
    norm = np.linalg.norm(H, ord=2)
    if norm == 0:
        return None
    return H / norm

#### Find phase angles

In [17]:
# Compute QSVT phase angles for Chebyshev polynomial T_degree(x)
def compute_phase_angles(degree, method="sym_qsp"):

    # Chebyshev basis representation: T_degree(x) = [0, 0, ..., 0, 1]
    # where 1 is at position 'degree' (representing T_degree in Chebyshev basis)
    cheb_coeffs = np.zeros(degree + 1)
    cheb_coeffs[degree] = 1.0

    # Compute phase angles using sym_qsp method with Chebyshev basis (recommended)
    # This is the fastest and most stable method for Chebyshev polynomials
    try:
        if method == "sym_qsp":
            # sym_qsp returns (full_phi, reduced_phi, parity)
            phases_qsp, _, _ = QuantumSignalProcessingPhases(
                cheb_coeffs,
                method="sym_qsp",
                chebyshev_basis=True,
                signal_operator="Wx"
            )
        else:
            # For other methods, use standard approach
            phases_qsp = QuantumSignalProcessingPhases(
                cheb_coeffs,
                method=method,
                signal_operator="Wx"
            )
    except Exception as e:
        # Fallback to laurent method if sym_qsp fails
        if method != "laurent":
            print(f"Warning: {method} method failed, trying laurent method")
            try:
                phases_qsp = QuantumSignalProcessingPhases(
                    cheb_coeffs,
                    method="laurent",
                    signal_operator="Wx"
                )
            except Exception as e2:
                raise RuntimeError(f"Both methods failed. Original: {e}, Fallback: {e2}")
        else:
            raise

    # Convert PyQSP phases to PennyLane QSVT format
    phases_qsvt = qml.transform_angles(phases_qsp, "QSP", "QSVT")
    return np.array(phases_qsvt)


In [18]:
# Test on random sample: normalize and verify
try:
    # Filter to only molecules with π-systems
    qm9_with_pi = qm9_huckel[qm9_huckel['pi_atoms'].apply(lambda x: len(ast.literal_eval(x)) > 0)]
    
    # Take a single random sample from filtered DataFrame
    sample_row = qm9_with_pi.sample(n=1).iloc[0]
    
    smiles_sample = sample_row['smiles']
    
    # Parse the H_pi matrix from the DataFrame
    H_pi_str = sample_row['H_pi']
    pi_atoms_str = sample_row['pi_atoms']
    
    # Convert string representations to numpy array and list
    # Note: H_pi is stored in NumPy's array string format, not Python literal format
    # We need to fix the format by adding commas between rows
    H_pi_str_fixed = H_pi_str.replace(']  [', '], [').replace('] [', '], [')
    H_pi_sample = np.array(ast.literal_eval(H_pi_str_fixed))
    pi_atoms_sample = ast.literal_eval(pi_atoms_str)
    
    # Normalize the Hamiltonian
    H_norm_sample = normalize_hamiltonian(H_pi_sample)
    
    print(f"Sample molecule SMILES: {smiles_sample}")
    print(f"π-system size: {len(pi_atoms_sample)}")
    print(f"Original ||H||_2: {np.linalg.norm(H_pi_sample, ord=2):.6f}")
    print(f"Normalized ||H||_2: {np.linalg.norm(H_norm_sample, ord=2):.6f}")
    print(f"Matrix shape: {H_norm_sample.shape}")
except NameError as e:
    print(f"Required data not available. Make sure qm9_huckel DataFrame is loaded properly.")


Sample molecule SMILES: CC1=CC(F)=CC(F)=N1
π-system size: 6
Original ||H||_2: 2.107446
Normalized ||H||_2: 1.000000
Matrix shape: (6, 6)


In [19]:
# Compute phase angles for degree-2 Chebyshev polynomial
try:
    degree = 2
    phases = compute_phase_angles(degree)
    print(f"Degree {degree} Chebyshev polynomial")
    print(f"Number of phase angles: {len(phases)}")
    print(f"Phase angles: {phases}")
except (ImportError, NameError) as e:
    print(f"Error: {e}")
    print("Make sure pyqsp is installed: pip install pyqsp")


[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 1.585e-01
iter: 002 --- err: 3.823e-02
iter: 003 --- err: 9.479e-03
iter: 004 --- err: 2.365e-03
iter: 005 --- err: 5.910e-04
iter: 006 --- err: 1.477e-04
iter: 007 --- err: 3.693e-05
iter: 008 --- err: 9.233e-06
iter: 009 --- err: 2.308e-06
iter: 010 --- err: 5.770e-07
iter: 011 --- err: 1.443e-07
iter: 012 --- err: 3.606e-08
iter: 013 --- err: 9.016e-09
iter: 014 --- err: 2.254e-09
iter: 015 --- err: 5.635e-10
iter: 016 --- err: 1.409e-10
iter: 017 --- err: 3.522e-11
iter: 018 --- err: 8.805e-12
iter: 019 --- err: 2.201e-12
iter: 020 --- err: 5.504e-13
[sym_qsp] Stop criteria satisfied.
Degree 2 Chebyshev polynomial
Number of phase angles: 3
Phase angles: [-6.28318557e+00  1.57079633e+00 -2.62271606e-07]


#### pad matrix

In [21]:
def pad_matrix_to_power_of_2(H):
    """Pad matrix to next power of 2 for QSVT (required for some implementations)."""
    if H is None:
        return None
    n = H.shape[0]
    next_power = 2 ** int(np.ceil(np.log2(n)))
    if next_power == n:
        return H
    padded = np.zeros((next_power, next_power), dtype=H.dtype)
    padded[:n, :n] = H
    return padded


#### Build quantum circuit

In [22]:
def build_qsvt_circuit(H_norm, phases, n_qubits):
    """Build QSVT circuit for spectral filtering.

    Args:
        H_norm: Normalized Hamiltonian matrix (n×n)
        phases: Phase angles for QSVT
        n_qubits: Matrix dimension (used for compatibility, actual qubits calculated from H_norm)

    Note: BlockEncode requires qubits based on the matrix dimension. For an n×n matrix,
    BlockEncode needs enough qubits to represent a Hilbert space of at least 2n×2n.
    """
    matrix_dim = H_norm.shape[0]

    # BlockEncode requires qubits such that 2^num_qubits >= 2*matrix_dim
    # For an 8×8 matrix, we need 2^4 = 16 >= 16, so 4 qubits
    # Formula: num_qubits = ceil(log2(2*matrix_dim))
    required_qubits = int(np.ceil(np.log2(2 * matrix_dim)))

    # Verify: 2^required_qubits should be >= 2*matrix_dim
    if 2**required_qubits < 2 * matrix_dim:
        required_qubits += 1

    print(f"  Matrix dimension: {matrix_dim}×{matrix_dim}")
    print(f"  Required qubits for BlockEncode: {required_qubits} (Hilbert space: {2**required_qubits}×{2**required_qubits})")

    # Create block encoding with appropriate number of qubits
    block_encoding = qml.BlockEncode(H_norm, wires=range(required_qubits))

    # Projectors use the same qubits
    projectors = [qml.PCPhase(phi, dim=2, wires=range(required_qubits)) for phi in phases]

    # Device needs enough qubits: data qubits + ancilla qubits for QSVT
    # QSVT typically needs 3 additional ancilla qubits
    total_qubits = required_qubits + 3
    dev = qml.device("default.qubit", wires=total_qubits)

    @qml.qnode(dev)
    def qsvt_circuit(input_state=None):
        if input_state is not None:
            qml.StatePrep(input_state, wires=range(required_qubits))

        qml.QSVT(block_encoding, projectors)
        return qml.state()

    return qsvt_circuit


In [36]:
# Use the normalized Hamiltonian from the sample molecule (Cell 8)
H_padded = pad_matrix_to_power_of_2(H_norm_sample)

print(f"Matrix: {H_norm_sample.shape} → {H_padded.shape} (padded)")
print(f"Phase angles: {len(phases)}")
    
qsvt_circuit = build_qsvt_circuit(H_padded, phases, H_padded.shape[0])
filtered_state = qsvt_circuit()
    
print(f"QSVT executed successfully")
print(f"State shape: {filtered_state.shape}, norm: {np.linalg.norm(filtered_state):.6f}")



Matrix: (6, 6) → (8, 8) (padded)
Phase angles: 3
  Matrix dimension: 8×8
  Required qubits for BlockEncode: 4 (Hilbert space: 16×16)
QSVT executed successfully
State shape: (128,), norm: 1.000000


In [37]:
def extract_z_expectations(H_norm, phases, n_qubits):
    """Extract single-qubit Z expectation values after QSVT.

    Note: QSVT output state includes ancilla qubits. We measure on the data qubits
    which are the first n_qubits wires after QSVT application.
    """
    # Calculate required qubits for BlockEncode
    matrix_dim = H_norm.shape[0]
    required_qubits = int(np.ceil(np.log2(2 * matrix_dim)))
    if 2**required_qubits < 2 * matrix_dim:
        required_qubits += 1

    # Total qubits: data qubits + ancilla for QSVT
    total_qubits = required_qubits + 3
    dev = qml.device("default.qubit", wires=total_qubits)

    block_encoding = qml.BlockEncode(H_norm, wires=range(required_qubits))
    projectors = [qml.PCPhase(phi, dim=2, wires=range(required_qubits)) for phi in phases]

    z_expectations = []
    # Measure on data qubits (first required_qubits wires)
    for i in range(min(n_qubits, required_qubits)):
        @qml.qnode(dev)
        def measure_z_i():
            qml.QSVT(block_encoding, projectors)
            # Trace out ancilla qubits and measure on data qubit i
            return qml.expval(qml.PauliZ(i))
        z_expectations.append(measure_z_i())

    return np.array(z_expectations)


In [38]:
def extract_zz_correlations(H_norm, phases, bonded_pairs, n_qubits):
    """Extract ZZ correlations for bonded pairs after QSVT."""
    # Calculate required qubits for BlockEncode (same logic as build_qsvt_circuit)
    matrix_dim = H_norm.shape[0]
    required_qubits = int(np.ceil(np.log2(2 * matrix_dim)))
    if 2**required_qubits < 2 * matrix_dim:
        required_qubits += 1

    block_encoding = qml.BlockEncode(H_norm, wires=range(required_qubits))
    projectors = [qml.PCPhase(phi, dim=2, wires=range(required_qubits)) for phi in phases]
    dev = qml.device("default.qubit", wires=required_qubits + 3)

    zz_correlations = []
    for i, j in bonded_pairs:
        # Ensure indices are within valid range
        if i < required_qubits and j < required_qubits:
            @qml.qnode(dev)
            def measure_zz_ij():
                qml.QSVT(block_encoding, projectors)
                return qml.expval(qml.PauliZ(i) @ qml.PauliZ(j))
            zz_correlations.append(measure_zz_ij())

    return np.array(zz_correlations)


In [39]:
# Sample a random molecule from the QM9 dataframe with π-system
# Filter to only molecules with π-systems
qm9_with_pi = qm9_huckel[qm9_huckel['pi_atoms'].apply(lambda x: len(ast.literal_eval(x)) > 0)]

# Take a single random sample from filtered DataFrame
sample_row = qm9_with_pi.sample(n=1).iloc[0]
smiles_simple = sample_row['smiles']

# Parse the H_pi matrix from the DataFrame (using CSV data)
H_pi_str = sample_row['H_pi']
pi_atoms_str = sample_row['pi_atoms']

# Convert string representations to numpy array and list
# Note: H_pi is stored in NumPy's array string format, not Python literal format
# We need to fix the format by adding commas between rows
H_pi_str_fixed = H_pi_str.replace(']  [', '], [').replace('] [', '], [')
H_pi_simple = np.array(ast.literal_eval(H_pi_str_fixed))
pi_atoms_simple = ast.literal_eval(pi_atoms_str)

# Normalize the Hamiltonian
H_norm_simple = normalize_hamiltonian(H_pi_simple)

# Extract features from simple case
n_qubits_simple = int(np.log2(H_norm_simple.shape[0]))
phases_simple = compute_phase_angles(degree=2)
z_features = extract_z_expectations(H_norm_simple, phases_simple, n_qubits_simple)
zz_features = extract_zz_correlations(H_norm_simple, phases_simple, [(0, 1)], n_qubits_simple)

print(f"Sample molecule SMILES: {smiles_simple}")
print(f"π-system size: {len(pi_atoms_simple)}")
print(f"Matrix shape: {H_norm_simple.shape}")
print(f"Z expectations: {z_features}")
print(f"ZZ correlations: {zz_features}")

# Verify: Check what's happening
print(f"\nVerification:")
eigvals = np.linalg.eigvals(H_norm_simple)
print(f"Matrix eigenvalues: {eigvals}")

# For T_2(x) = 2x² - 1, check what it maps eigenvalues to
T2_eigvals = 2 * eigvals**2 - 1
print(f"T_2(eigenvalues) = 2*λ² - 1: {T2_eigvals}")

# Note: Z=1.0 is correct if state is |0⟩
# QSVT without input state starts from |0⟩, so measurements on |0⟩ give Z=1
# Both eigenvalues map to 1.0 under T_2, which is expected
print(f"\nNote: Values of 1.0 are expected when measuring |0⟩ state.")
print(f"T_2 maps both eigenvalues (±1) to 1.0, so this is consistent.")


[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 1.585e-01
iter: 002 --- err: 3.823e-02
iter: 003 --- err: 9.479e-03
iter: 004 --- err: 2.365e-03
iter: 005 --- err: 5.910e-04
iter: 006 --- err: 1.477e-04
iter: 007 --- err: 3.693e-05
iter: 008 --- err: 9.233e-06
iter: 009 --- err: 2.308e-06
iter: 010 --- err: 5.770e-07
iter: 011 --- err: 1.443e-07
iter: 012 --- err: 3.606e-08
iter: 013 --- err: 9.016e-09
iter: 014 --- err: 2.254e-09
iter: 015 --- err: 5.635e-10
iter: 016 --- err: 1.409e-10
iter: 017 --- err: 3.522e-11
iter: 018 --- err: 8.805e-12
iter: 019 --- err: 2.201e-12
iter: 020 --- err: 5.504e-13
[sym_qsp] Stop criteria satisfied.
Sample molecule SMILES: OC1CC2CC12CC#C
π-system size: 2
Matrix shape: (2, 2)
Z expectations: [1.]
ZZ correlations: [1.]

Verification:
Matrix eigenvalues: [ 1. -1.]
T_2(eigenvalues) = 2*λ² - 1: [1. 1.]

Note: Values of 1.0 are expected when measuring |0⟩ state.
T_2 maps both eigenvalues (±1) to 1.0, so this is consis

#### Feature Extraction

In [131]:
def extract_qsvt_features(mol, degree=2, pi_atoms_list=None):
    """Extract QSVT features for a molecule: Z expectations and ZZ correlations."""
    H_pi, pi_atoms = build_pi_huckel_hamiltonian(mol)
    H_norm = normalize_hamiltonian(H_pi)

    if H_norm is None or H_norm.size == 0:
        return None, None, None

    n = H_norm.shape[0]
    if not (n > 0 and (n & (n - 1)) == 0):
        return None, None, None

    n_qubits = int(np.log2(n))
    phases = compute_phase_angles(degree)

    z_features = extract_z_expectations(H_norm, phases, n_qubits)

    bonded_pairs = []
    if pi_atoms_list is not None:
        for bond in mol.GetBonds():
            begin_idx = bond.GetBeginAtomIdx()
            end_idx = bond.GetEndAtomIdx()
            if begin_idx in pi_atoms and end_idx in pi_atoms:
                i = pi_atoms.index(begin_idx)
                j = pi_atoms.index(end_idx)
                if i < j and i < n_qubits and j < n_qubits:
                    bonded_pairs.append((i, j))

    if not bonded_pairs:
        bonded_pairs = [(i, i+1) for i in range(n_qubits-1)]

    zz_features = extract_zz_correlations(H_norm, phases, bonded_pairs, n_qubits)

    return z_features, zz_features, pi_atoms


In [132]:
def chebyshev_filter_classical(H, degree):
    """Apply Chebyshev polynomial filter classically using recurrence relation."""
    if H is None or H.size == 0:
        return None

    n = H.shape[0]
    T_prev = np.eye(n)
    T_curr = H

    result = T_prev + T_curr

    for k in range(2, degree + 1):
        T_next = 2 * H @ T_curr - T_prev
        result += T_next
        T_prev, T_curr = T_curr, T_next

    return result


In [1]:
# Advanced test: Benzene with different input states and polynomial degrees
print("=" * 70)
print("Advanced QSVT Test: Benzene (6×6 matrix)")
print("=" * 70)

# Get benzene Hamiltonian
try:
    smiles_benzene = "c1ccccc1"
    mol_benzene = Chem.MolFromSmiles(smiles_benzene)
    mol_benzene = Chem.AddHs(mol_benzene)
    Chem.SanitizeMol(mol_benzene)

    H_pi_benzene, pi_atoms_benzene = build_pi_huckel_hamiltonian(mol_benzene)
    H_norm_benzene = normalize_hamiltonian(H_pi_benzene)
    H_padded_benzene = pad_matrix_to_power_of_2(H_norm_benzene)

    print(f"\nBenzene π-system:")
    print(f"  Atoms: {len(pi_atoms_benzene)}")
    print(f"  Matrix: {H_norm_benzene.shape} → {H_padded_benzene.shape} (padded)")

    eigvals_benzene = np.linalg.eigvals(H_norm_benzene)
    print(f"  Eigenvalues: {eigvals_benzene}")
    print(f"  Eigenvalue range: [{eigvals_benzene.min():.4f}, {eigvals_benzene.max():.4f}]")

    # Test different polynomial degrees
    degrees = [2, 4]
    results = {}

    for degree in degrees:
        print(f"\n{'='*70}")
        print(f"Testing degree-{degree} Chebyshev polynomial")
        print(f"{'='*70}")

        phases_deg = compute_phase_angles(degree)
        print(f"Phase angles: {len(phases_deg)} angles")

        # Build QSVT circuit
        qsvt_circuit_benzene = build_qsvt_circuit(H_padded_benzene, phases_deg, H_padded_benzene.shape[0])

        # Get output state
        state_benzene = qsvt_circuit_benzene()
        print(f"Output state shape: {state_benzene.shape}")
        print(f"State norm: {np.linalg.norm(state_benzene):.6f}")

        # Calculate required qubits for measurements
        matrix_dim = H_padded_benzene.shape[0]
        required_qubits = int(np.ceil(np.log2(2 * matrix_dim)))
        if 2**required_qubits < 2 * matrix_dim:
            required_qubits += 1

        # Extract features
        n_data_qubits = int(np.log2(H_norm_benzene.shape[0]))
        z_features_benzene = extract_z_expectations(H_padded_benzene, phases_deg, n_data_qubits)

        # Get bonded pairs for benzene (ring structure)
        bonded_pairs_benzene = []
        for i in range(n_data_qubits):
            bonded_pairs_benzene.append((i, (i+1) % n_data_qubits))  # Ring structure

        zz_features_benzene = extract_zz_correlations(H_padded_benzene, phases_deg, bonded_pairs_benzene[:3], n_data_qubits)

        results[degree] = {
            'z': z_features_benzene,
            'zz': zz_features_benzene,
            'eigvals': eigvals_benzene
        }

        print(f"\nFeatures (degree {degree}):")
        print(f"  Z expectations: {z_features_benzene}")
        print(f"  ZZ correlations (first 3 pairs): {zz_features_benzene}")
        print(f"  Z mean: {z_features_benzene.mean():.4f}, std: {z_features_benzene.std():.4f}")

        # Show what T_degree maps eigenvalues to
        T_deg_eigvals = []
        for lam in eigvals_benzene:
            # T_2(x) = 2x² - 1, T_4(x) = 8x⁴ - 8x² + 1
            if degree == 2:
                T_val = 2 * lam**2 - 1
            elif degree == 4:
                T_val = 8 * lam**4 - 8 * lam**2 + 1
            else:
                # Use numpy Chebyshev polynomial
                from numpy.polynomial.chebyshev import Chebyshev
                cheb_coeffs = np.zeros(degree + 1)
                cheb_coeffs[degree] = 1.0
                T_poly = Chebyshev(cheb_coeffs)
                T_val = T_poly(lam)
            T_deg_eigvals.append(T_val)

        print(f"  T_{degree}(eigenvalues): {np.array(T_deg_eigvals)}")
        print(f"  T_{degree} range: [{min(T_deg_eigvals):.4f}, {max(T_deg_eigvals):.4f}]")

    # Compare results
    print(f"\n{'='*70}")
    print("Comparison: Degree 2 vs Degree 4")
    print(f"{'='*70}")
    print(f"Degree 2 - Z mean: {results[2]['z'].mean():.4f}, std: {results[2]['z'].std():.4f}")
    print(f"Degree 4 - Z mean: {results[4]['z'].mean():.4f}, std: {results[4]['z'].std():.4f}")
    print(f"\nDegree 2 - ZZ mean: {results[2]['zz'].mean():.4f}")
    print(f"Degree 4 - ZZ mean: {results[4]['zz'].mean():.4f}")

    # Additional analysis: Show eigenvalue filtering effect
    print(f"\n{'='*70}")
    print("Eigenvalue Filtering Analysis (Benzene)")
    print(f"{'='*70}")
    print("Original eigenvalues:", eigvals_benzene)

    # Calculate what T_2 and T_4 actually map to
    T2_benzene = 2 * eigvals_benzene**2 - 1
    T4_benzene = 8 * eigvals_benzene**4 - 8 * eigvals_benzene**2 + 1

    print(f"\nT_2(eigenvalues): {T2_benzene}")
    print(f"T_4(eigenvalues): {T4_benzene}")
    print(f"\nNote: T_2 and T_4 happen to map these eigenvalues to the same values.")
    print("This is because T_2(±1) = T_4(±1) = 1 and T_2(±0.5) = T_4(±0.5) = -0.5")

    # Test with different molecule for more variation
    print(f"\n{'='*70}")
    print("Testing on Naphthalene (10×10 matrix) for more variation")
    print(f"{'='*70}")
    try:
        smiles_naph = "c1ccc2ccccc2c1"
        mol_naph = Chem.MolFromSmiles(smiles_naph)
        mol_naph = Chem.AddHs(mol_naph)
        Chem.SanitizeMol(mol_naph)

        H_pi_naph, pi_atoms_naph = build_pi_huckel_hamiltonian(mol_naph)
        H_norm_naph = normalize_hamiltonian(H_pi_naph)
        H_padded_naph = pad_matrix_to_power_of_2(H_norm_naph)

        eigvals_naph = np.linalg.eigvals(H_norm_naph)
        print(f"Naphthalene: {len(pi_atoms_naph)} atoms, matrix {H_norm_naph.shape} → {H_padded_naph.shape}")
        print(f"Eigenvalues: {eigvals_naph}")
        print(f"Eigenvalue range: [{eigvals_naph.min():.4f}, {eigvals_naph.max():.4f}]")

        # Test degree 2
        phases_naph = compute_phase_angles(degree=2)
        n_data_qubits_naph = int(np.log2(H_norm_naph.shape[0]))
        z_features_naph = extract_z_expectations(H_padded_naph, phases_naph, n_data_qubits_naph)

        print(f"\nNaphthalene features (degree 2):")
        print(f"  Z expectations: {z_features_naph}")
        print(f"  Z mean: {z_features_naph.mean():.4f}, std: {z_features_naph.std():.4f}")
        print(f"  Z range: [{z_features_naph.min():.4f}, {z_features_naph.max():.4f}]")

        # Show T_2 transformation
        T2_naph = 2 * eigvals_naph**2 - 1
        print(f"  T_2(eigenvalues): {T2_naph}")
        print(f"  T_2 range: [{T2_naph.min():.4f}, {T2_naph.max():.4f}]")

        # Compare benzene vs naphthalene
        print(f"\n{'='*70}")
        print("Benzene vs Naphthalene Comparison")
        print(f"{'='*70}")
        print(f"Benzene: 6 atoms, Z mean={results[2]['z'].mean():.4f}, std={results[2]['z'].std():.4f}")
        print(f"Naphthalene: 10 atoms, Z mean={z_features_naph.mean():.4f}, std={z_features_naph.std():.4f}")
        print(f"\n✓ Naphthalene shows more variation (std={z_features_naph.std():.4f} vs {results[2]['z'].std():.4f})")
        print(f"✓ This demonstrates QSVT can extract different features for different molecules!")

    except Exception as e:
        print(f"Could not test naphthalene: {e}")

except NameError as e:
    print(f"Error: {e}")
    print("Make sure RDKit and build_pi_huckel_hamiltonian are available.")
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()


Advanced QSVT Test: Benzene (6×6 matrix)
Error: name 'Chem' is not defined
Make sure RDKit and build_pi_huckel_hamiltonian are available.


In [ ]:
# Run QSVT feature extraction on multiple molecules from QM9
# How many molecules to process (adjust as needed)
max_molecules = 100

qsvt_records = []

for idx, row in qm9.head(max_molecules).iterrows():
    smiles = row["smiles"] if "smiles" in row else row["SMILES"]
    try:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        Chem.SanitizeMol(mol)

        # Extract QSVT features (degree-2 Chebyshev by default)
        z_feat, zz_feat, pi_atoms = extract_qsvt_features(mol, degree=2)

        # Skip molecules where feature extraction failed (e.g., non power-of-2 size)
        if z_feat is None or zz_feat is None:ow m
            continue

        qsvt_records.append({
            "index": idx,
            "smiles": smiles,
            "num_pi_atoms": len(pi_atoms),
            "z_features": z_feat,
            "zz_features": zz_feat,
        })
    except Exception as e:
        # Skip problematic molecules
        print(f"Skipping {smiles}: {e}")
        continue

qm9_qsvt_features = pd.DataFrame(qsvt_records)
print(f"Computed QSVT features for {len(qsvt_records)} molecules (out of {max_molecules}).")
qm9_qsvt_features.head()


[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 1.585e-01
iter: 002 --- err: 3.823e-02
iter: 003 --- err: 9.479e-03
iter: 004 --- err: 2.365e-03
iter: 005 --- err: 5.910e-04
iter: 006 --- err: 1.477e-04
iter: 007 --- err: 3.693e-05
iter: 008 --- err: 9.233e-06
iter: 009 --- err: 2.308e-06
iter: 010 --- err: 5.770e-07
iter: 011 --- err: 1.443e-07
iter: 012 --- err: 3.606e-08
iter: 013 --- err: 9.016e-09
iter: 014 --- err: 2.254e-09
iter: 015 --- err: 5.635e-10
iter: 016 --- err: 1.409e-10
iter: 017 --- err: 3.522e-11
iter: 018 --- err: 8.805e-12
iter: 019 --- err: 2.201e-12
iter: 020 --- err: 5.504e-13
[sym_qsp] Stop criteria satisfied.
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 1.585e-01
iter: 002 --- err: 3.823e-02
iter: 003 --- err: 9.479e-03
iter: 004 --- err: 2.365e-03
iter: 005 --- err: 5.910e-04
iter: 006 --- err: 1.477e-04
iter: 007 --- err: 3.693e-05
iter: 008 --- err: 9.233e-06
iter: 009 --- err: 

,index,smiles,num_pi_atoms,z_features,zz_features
0,3,C#C,2,[0.9999999999999999],[]
1,4,C#N,2,[-0.4160261391251687],[]
2,5,C=O,2,[0.026755361358055618],[]
3,8,CC#C,2,[0.9999999999999999],[]
4,9,CC#N,2,[-0.4160261391251687],[]
